## Classes and Interfaces
### Item 37: Compose Classes Instead of Nesting Many Levels of Built-in Types
- Avoid making dictionaries with values that are dictionaries, long tuples, or complex nesting of other built-in types.
- Use `namedtuple` for lightweight, immutable data containers before you need the flexibility of a full class.
- Move your bookkeeping code to using multiple classes when your internal state dictionaries get complicated.
#### Example
We want to record the grades of a set of students whose names aren't known in advance. A class can hold the names in a dictionary instead of using a predefined attribute for each student.

In [2]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)
    
book = SimpleGradebook()
book.add_student("Sarah Brook")
book.report_grade("Sarah Brook", 90)
book.report_grade("Sarah Brook", 85)
book.report_grade("Sarah Brook", 95)
print(book.average_grade("Sarah Brook"))


90.0


- Dictionaries and their related built-in types are so easy to use that they have the risk of overextending them to write brittle code. 
- If we want to extend the `SimpleGradebook` class to keep a list of grades by subject, not just overall. This can be done by changing the `_grades` dictionary to hold yet another dictionary that maps the subject name to the grades. 
- The inner dictionary can be a `defaultdict` instance to handle missing subjects.

In [3]:
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {} # Outer dict
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # Inner dict

This seems straightforward, the `report_grade` and `average_grade` methods gain a bit of complexity to deal with the multilevel dictionary. If the requirements change again and we also need to track the weight of each score toward the overall grade in the class, it gets even more complex.
We can make the grades be a tuple and map the subject in the inner dict to a tuple of `(score, weight)`. 

In [4]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))


Now the `average_grade` method now has to loop within a loop and is difficult to read:

In [6]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))

    def average_grade(self, name):
        by_subject = self._grades[name]

        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
        return score_sum / score_count

book = WeightedGradebook()
book.add_student("Sarah Brook")
book.report_grade("Sarah Brook", "Math", 75, 0.05)
book.report_grade("Sarah Brook", "Math", 65, 0.15)
book.report_grade("Sarah Brook", "Math", 70, 0.8)
book.report_grade("Sarah Brook", "Gym", 100, 0.4)
book.report_grade("Sarah Brook", "Gym", 85, 0.6)

print(book.average_grade("Sarah Brook"))


80.25


#### Refactoring to Classes
We can start moving to classes at the bottom of the dependency tree: a single grade. A class seems too heavyweight for such a simple information. A `tuple` though, seems appropriate because grades are immutable. The problem with that is that `tuple` instances are positional. For example, if we want to associate more information with grade than its weight, such as a set of notes from the teacher, we need to rewrite every usage of the two-tuple to be aware that there are now three items present instead of two. 

As soon as you find yourself going longer than a two-tuple, it's time to consider another approach. The `namedtuple` type in the `collections` built-in module does that: It lets you easily define a tiny, immutable data class:

In [11]:
from collections import namedtuple

Grade = namedtuple("Grade", ("score", "weight"))

The fields are accessible with named attributes. Having named attributes makes it easy to move from a `namedtuple` to a class later if the requirements change again and we need support mutability or behaviours in the simple data containers for example.

#### Limitations of `namedtuple`
1. You can't specify default argument values for `namedtuple` classes. If you find yourself using more than a handful of attibutes, using the built-in `dataclasses` module may be a better choice.
2. The attribute values of the `namedtuple` instances are still accessible via numerical indexes and iteration. Especially in externalised APIs, this can lead to unintentional usage that makes it harder to move to a real class later. If you're not in control of all of the usage of your `namedtuple` instances, it's better to explicitly define a new class.
Now we can write a class to represent a single subject that contains a set of grades:

In [7]:
class Subject:
    def __init__(self):
        self._grades = []
    
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight


Now, we can define a class that represents a set of subjects studied by a single student.

In [12]:
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
    
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

And finally, we will define a container for all the students, keyed dynamically by their names:

In [13]:
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)
    
    def get_student(self, name):
        return self._students[name]


book = Gradebook()
albert = book.get_student("Albert Einstein")
math = albert.get_subject("Math")
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.8)

gym = albert.get_subject("Gym")
gym.report_grade(100, 0.4)
gym.report_grade(85, 0.6)
print(albert.average_grade())

80.25


### Item 38: Accept Functions Instead of Classes for Simple Interfaces
- Instead of defining and instantiating classes, you can often simply use functions for simple interfaces between components in Python.
- References to functions and methods in Python are first class, meaning then can be used in expressions (like any other type).
- The `__call__` special method enables instances of a class to be called like plain Python functions.
- When you need a function to maintain state, consider defining a class that provides the `__call__` method instead of defining a stateful closure for easier readibility. Do not define a class without the `__call__` method because it is not immediately obvious to the reader what the purpose of the class is. 

In [1]:
from collections import defaultdict

current = {"green": 12, "blue": 3}

increments = [
    ("red", 5),
    ("blue", 17),
    ("orange", 9)
]

class BetterCountMissing:
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0
    
counter = BetterCountMissing()
result = defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount

assert counter.added == 2

### Item 39: Use @classmethod Polymorphism to Construct Objects Generically
Polymorphasim enables multiple classes in a hierarchy to implement their own unique versions of a method.  

### Item 40: Initialise Parent Classes with `super`
The simple way to initialise a parent class from a child class is to directly call the parent class's `__init__` method with the child instance:

In [3]:
class MyBaseClass:
    def __init__(self, value):
       self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5) 

This works fine for base class hierarchies but breaks in many cases. If a class inherits from multiple classes (something that should be avoided in general), calling the superclasses' `__init__` methods directly ca lead to unpredictable behaviour.
1. One problem is that the `__init__` call order isn't specified across all subclasses.

In [11]:
class TimesTwo:
    def __init__(self):
        self.value *= 2

class PlusFive:
    def __init__(self):
        self.value += 5

This class defines its parent classes in one oredering:

In [9]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

And constructing it produces a result that matches the parent class ordering:

In [12]:
foo = OneWay(5)
print("First ordering value is (5 * 2) + 5 = ", foo.value)

First ordering value is (5 * 2) + 5 =  15


Here's another class that defines the same parent classes but in a different ordering (`PlusFive` followed by `TimesTwo` instead of the other way around):

In [13]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

But the calls to the parent class constructors are in the same order as before, which means this class's behaviour doesn't match the order of the parent classes in its definition. The conflict here between the inheritance base classes and the `__init__` calls is hard to spot.